In [7]:
from sklearn.datasets import fetch_openml

from neural_blueprints.architectures import VariationalAutoEncoder
from neural_blueprints.config.architectures import AutoEncoderConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections import TabularProjectionConfig
from neural_blueprints.utils import Trainer, infer_types
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import TabularDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [8]:
data = fetch_openml(name="adult", version=2, as_frame=True)
X = data.data
y = data.target

data = X.copy()
data['income'] = y

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [9]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2026-01-22 17:09:47,718 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2026-01-22 17:09:47,718 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
2026-01-22 17:09:47,720 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'workclass' has 2799/5.73% NaN values; these will be encoded as 0.
2026-01-22 17:09:47,800 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'occupation' has 2809/5.75% NaN values; these will be encoded as 0.
2026-01-22 17:09:47,852 - neural_blueprints.preprocess.tabular_preprocess - INFO - Discrete column 'native-country' has 857/1.75% NaN values; these will be encoded as 0.


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,4,0.145129,2,14,5,7,4,3,2,0.000000,0.0,0.397959,39,1
1,0.287671,4,0.052451,12,16,3,5,1,5,2,0.000000,0.0,0.500000,39,1
2,0.150685,2,0.219649,8,4,3,11,1,5,2,0.000000,0.0,0.397959,39,2
3,0.369863,4,0.100153,16,2,3,7,1,3,2,0.076881,0.0,0.397959,39,2
4,0.013699,0,0.061708,16,2,5,0,4,5,1,0.000000,0.0,0.295918,39,1


### Dataset Reconstruction Accuracy

In [10]:
dataset = TabularDataset(
    data = data
)

train_dataset, val_dataset, test_dataset = dataset.random_split([0.9, 0.05, 0.05])

In [14]:
latent_dim = 32

vae_config = AutoEncoderConfig(
    input_spec=(len(dataset.cardinalities),),
    output_spec=(len(dataset.cardinalities),),
    input_projection=TabularProjectionConfig(
        input_cardinalities=dataset.cardinalities,
        hidden_dims=[64, 64],
        projection_dim=latent_dim,
        output_dim=[len(dataset.cardinalities)*latent_dim],
    ),
    output_projection=TabularProjectionConfig(
        input_dim=[len(dataset.cardinalities) * latent_dim],
        hidden_dims=[64, 64],
        output_cardinalities=dataset.cardinalities,
    ),
    encoder_layers= [len(dataset.cardinalities)*latent_dim, 512, 256, 128],
    symmetric= True,
    latent_dim= latent_dim,
    dropout_p= 0.1,
    normalization= 'batchnorm1d',
    activation= 'relu',
    final_activation= None,
)

model = VariationalAutoEncoder(vae_config)
model.blueprint(batch_size=256, with_graph=False)

2026-01-22 17:11:23,845 - neural_blueprints.architectures.autoencoder - INFO - Using input projection: TabularInputProjection
2026-01-22 17:11:23,858 - neural_blueprints.architectures.autoencoder - INFO - Using output projection: TabularOutputProjection


Layer (type:depth-idx)                                                      Output Shape              Param #
VariationalAutoEncoder                                                      [256, 1]                  --
├─TabularInputProjection: 1-1                                               [256, 480]                --
│    └─ModuleList: 2-1                                                      --                        --
│    │    └─NumericalProjection: 3-1                                        [256, 32]                 6,624
│    │    └─DiscreteProjection: 3-2                                         [256, 32]                 11,424
│    │    └─NumericalProjection: 3-3                                        [256, 32]                 6,624
│    │    └─DiscreteProjection: 3-4                                         [256, 32]                 11,872
│    │    └─DiscreteProjection: 3-5                                         [256, 32]                 11,872
│    │    └─DiscreteProjection: 

In [15]:
trainer = Trainer(
    config=TrainerConfig(
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=256,
        early_stopping_patience=5,
        criterion="vae_reconstruction",
        optimizer='adam'
    ),
    model=model
)
trainer.train(train_dataset, val_dataset, epochs=5)
trainer.predict(test_dataset)

2026-01-22 17:11:25,618 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu
Training Epochs:  20%|██        | 1/5 [00:04<00:18,  4.73s/epoch]

Epoch 1/5, Training Loss: 86.7746, Validation Loss: 20.2731


Training Epochs:  40%|████      | 2/5 [00:09<00:14,  4.72s/epoch]

Epoch 2/5, Training Loss: 23.5542, Validation Loss: 18.8044


Training Epochs:  60%|██████    | 3/5 [00:14<00:09,  4.73s/epoch]

...

Training Epochs: 100%|██████████| 5/5 [00:23<00:00,  4.73s/epoch]
2026-01-22 17:11:49,254 - neural_blueprints.utils.trainer - INFO - Training completed in 23.63 seconds.
2026-01-22 17:11:49,254 - neural_blueprints.utils.trainer - INFO - Best validation loss: 1.8269e+01


Epoch 5/5, Training Loss: 18.7973, Validation Loss: 18.2690


2026-01-22 17:11:49,362 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 1.
2026-01-22 17:11:49,364 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 7.
2026-01-22 17:11:49,365 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 8.
2026-01-22 17:11:49,365 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 9.
2026-01-22 17:11:49,366 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 13.
2026-01-22 17:11:49,367 - neural_blueprints.utils.predict - WARNING - Warning: Only one class predicted for feature column 14.
2026-01-22 17:11:49,367 - neural_blueprints.utils.trainer - INFO - Inference completed in 0.02 seconds.


Feature Column 0:
Predicted attribute values: [0.28038183 0.29763067 0.27921906 0.28034574 0.28998032]
True attribute values: [0.09589041 0.12328767 0.5068493  0.60273975 0.369863  ]
Accuracy: 0.1998

Feature Column 1:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [4. 4. 4. 4. 4.]
Accuracy: 0.6852

Feature Column 2:
Predicted attribute values: [0.12496077 0.11182008 0.12407087 0.12708023 0.11761384]
True attribute values: [0.24488486 0.05324011 0.22203347 0.06035728 0.13069957]
Accuracy: 0.5534

Feature Column 3:
Predicted attribute values: [12 16 12 12 16]
True attribute values: [12.  9. 12.  9. 10.]
Accuracy: 0.2771

Feature Column 4:
Predicted attribute values: [16 16 16 16 16]
True attribute values: [16.  3. 16.  3.  5.]
Accuracy: 0.3279

Feature Column 5:
Predicted attribute values: [3 3 3 3 3]
True attribute values: [6. 1. 6. 3. 3.]
Accuracy: 0.4568

Feature Column 6:
Predicted attribute values: [4 4 4 4 4]
True attribute values: [ 8.  1. 12.  1. 12.]
Accuracy: 0

{'avg_discrete_accuracy': np.float64(0.5445763405648792),
 'avg_continuous_accuracy': np.float64(0.643389275480966),
 'overall_avg_accuracy': np.float64(0.5775139855369081)}